In [ ]:
!pip install sklearn

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q /content/drive/MyDrive/Dataset.zip -d /content/Datasets/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace /content/Datasets/Dataset/test/photo/0000000_jpg.rf.0149b611119f0b2fdad1269a492f75b7.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_path="/content/Datasets/Dataset/train"
test_path="/content/Datasets/Dataset/test"
val_path="/content/Datasets/Dataset/val"

In [ ]:
# re-size all the images 
IMAGE_SIZE = [224, 224]

In [ ]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)

In [ ]:
x_test=[]

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)
    

In [ ]:
x_val=[]

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)
    

In [ ]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [ ]:
train_x.shape,test_x.shape,val_x.shape

((2344, 224, 224, 3), (468, 224, 224, 3), (476, 224, 224, 3))

In [ ]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')


Found 2344 images belonging to 2 classes.
Found 468 images belonging to 2 classes.
Found 476 images belonging to 2 classes.


In [ ]:
training_set.class_indices

{'photo': 0, 'sign': 1}

In [ ]:
train_y=training_set.classes

In [ ]:
test_y=test_set.classes

In [ ]:
val_y=val_set.classes

In [ ]:
train_y.shape,test_y.shape,val_y.shape

((2344,), (468,), (476,))

In [ ]:
#preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)

prediction = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

#Early stopping to avoid overfitting of model

In [ ]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=50,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

Epoch 1/50
74/74 [==============================] - 48s 491ms/step - loss: 0.0268 - accuracy: 0.9872 - val_loss: 1.8062e-05 - val_accuracy: 1.0000
Epoch 2/50
74/74 [==============================] - 30s 409ms/step - loss: 1.5508e-05 - accuracy: 1.0000 - val_loss: 1.5660e-05 - val_accuracy: 1.0000
Epoch 3/50
74/74 [==============================] - 30s 409ms/step - loss: 1.3604e-05 - accuracy: 1.0000 - val_loss: 1.3505e-05 - val_accuracy: 1.0000
Epoch 4/50
74/74 [==============================] - 30s 407ms/step - loss: 1.1989e-05 - accuracy: 1.0000 - val_loss: 1.1679e-05 - val_accuracy: 1.0000
Epoch 5/50
74/74 [==============================] - 30s 407ms/step - loss: 1.0638e-05 - accuracy: 1.0000 - val_loss: 1.0343e-05 - val_accuracy: 1.0000
Epoch 6/50
74/74 [==============================] - 30s 407ms/step - loss: 9.5160e-06 - accuracy: 1.0000 - val_loss: 9.1006e-06 - val_accuracy: 1.0000
Epoch 7/50
 5/74 [=>............................] - ETA: 23s - loss: 7.6426e-06 - accuracy: 1.0000

In [ ]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()

plt.savefig('vgg-loss-rps-1.png')
plt.show()

In [ ]:
# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()

plt.savefig('vgg-acc-rps-1.png')
plt.show()

In [ ]:
model.evaluate(test_x,test_y,batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np

In [ ]:
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
accuracy_score(y_pred,test_y)

In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
confusion_matrix(y_pred,test_y)

In [ ]:
path="rps-results"
for img in os.listdir(path):
    img=image.load_img(path+"/"+img,target_size=(224,224))
    plt.imshow(img)
    plt.show()
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    images=np.vstack([x])
    pred=model.predict(images,batch_size=1) 
    if pred[0][0]>0.5:
        print("photo")
    elif pred[0][1]>0.5:
        print("sign")


In [ ]:
model.save("vgg-rps-final.h5")